![](../graphics/microsoftlogo.png)

# Workshop: Unlocking AI Potential for the Data Professional - Azure OpenAI

#### <i>A Microsoft Course from Microsoft Engineering and the FastTrack Team</i>

<p style="border-bottom: 1px solid lightgrey;"></p>

<img style="float: left; margin: 0px 15px 15px 0px;" src="https://raw.githubusercontent.com/microsoft/sqlworkshops/master/graphics/textbubble.png"> <h2>Course Notebook: Module 3</h2>

Welcome to this Microsoft solutions workshop on [*Unlocking AI Potential for the Data Professional with Azure OpenAI*](https://github.com/sqlserverworkshops/OpenAI-DataPro/tree/main). In this Notebook, you'll apply the concepts you learned in this Module.


Mastering the fundamentals and core concepts of OpenAI is indispensable for unlocking its full potential within Azure. Developers need a solid grasp of deploying Azure services like Azure OpenAI and Azure Cognitive Search, ensuring secure deployment aligned with responsible AI practices. This foundational knowledge guarantees that applications built on these services are not only functional but also ethically sound.

Moreover, proficiency in interacting with OpenAI's Large Language Models (LLMs) via REST API is crucial for seamless integration into diverse applications. Developers must understand the underlying principles of OpenAI, including selecting the right model for specific tasks and discerning the advantages of different models in terms of token utilization and response precision. Understanding prompts, completions, chats and overall prompt engineering best practices is pivotal for crafting top-notch interactions with Azure OpenAI, whether it involves generating summaries, translations, or other functions.

Furthermore, advanced concepts like smart load balancing for OpenAI endpoints and fine-tuning models are essential for maximizing performance and ensuring application resilience. As AI applications become increasingly intricate, grasping these core concepts not only empowers developers to construct more efficient solutions but also aids in mitigating potential risks associated with sophisticated language models. By simplifying the understanding of how these core concepts interconnect, our goal is to enable developers to build smarter applications using Azure OpenAI.

# 3.2 Create Azure OpenAI Environment

## Azure 

This notebook contains the script to create the necessary Azure environment to run the provided samples. The notebook uses [PowerShell](https://learn.microsoft.com/powershell/scripting/install/installing-powershell?view=powershell-7.3) and [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) to deploy all necessary Azure resources. Both tools are available on Windows, macOS and Linux environments.

## Configuration

This section performs two tasks:

- Deployment of necessary Azure Services (Azure OpenAI, Azure Cognitive Search) to run samples
- Store all necessary service endpoints, service API keys, Azure OpenAI deployment names in a centralized file (../01_DemoEnvironment/conf/application.env). This file is used by all notebooks in this repo to connect and authenticate against the deployed Azure services.

If you already have instances of Azure OpenAI and Azure Cognitive Search running you can rename the [configuration template](../conf/.env-sample) to `.env` and provide endpoint, API key and deployment names of a chat completion and an embedding model. We suggest to run the notebook to start a clean environment.

### Visual Studio Code

If you are running these steps below in Visual Studio Code make sure you switch your kernal to .NET Interactive so that it will run the PowerShell

## Step 1:   Login to Azure; Get, Set subscription

In [ ]:
# Check if you are already logged in
$loggedIn = az account show --query "name" -o tsv

if ($null -ne $loggedIn) {
    Write-Host "Already logged in as $loggedIn"
} else {
    Write-Host "Logging in..."
    az login
}
# Retrieve default subscription id
$subscriptionId = (
    (
        az account list -o json `
            --query "[?isDefault]"
    ) | ConvertFrom-Json
).id

# Set Subscription
az account set --subscription $subscriptionId
Write-Host "Subscription set to $subscriptionId"

Expected output:

If you're already logged in:
```
    Already logged in as xxxxx
    Subscription set to xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
```
If you aren't logged in a browser window will pop-up which allows you to log in
```
    Logging in...
    Subscription set to xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
```

## Step 2:   Define project unifier

The project unifier is used to allow multiple deployments of services which have a need for a unique custom endpoint.

In [ ]:
$random = Get-Random -Minimum 100 -Maximum 999

Write-Host "Unifier set to: $random"

Expected output:

```
Unifier set to: xxx
```

## Step 3:   Create Resource Group

In this sample all resources are deployed to `westus`. Feel free to change to your preferred location.

In [ ]:
$resourceGroup = "OpenAI-DataPro-RG"
$location = "westus"

az group create `
    --location $location `
    --resource-group $resourceGroup

Expected output: 

 `JSON string describing the newly created resource group`

## Step 4:   Create Azure OpenAI instance

An instance of Azure Cognitive Service with the kind `OpenAI` will be created. The `endpoint` and `API key` of the newly created instance are retrieved for later storage in the `application.env` file.

In [ ]:
$csOpenAIName = "aiservices$random"

az cognitiveservices account create `
    --name $csOpenAIName `
    --resource-group $resourceGroup `
    --location $location `
    --kind OpenAI `
    --sku S0 `
    --yes

$csOpenAIEndpoint = ( `
    az cognitiveservices account show `
        --name $csOpenAIName `
        --resource-group $resourceGroup `
        --query properties.endpoint `
        --output tsv `
)

$csOpenAIApiKey = (
    az cognitiveservices account keys list `
        --name $csOpenAIName `
        --resource-group $resourceGroup `
        --query key1 `
        --output tsv `
)

Expected output: 

 `JSON string describing the newly created Azure OpenAI instance`

## Step 5:   Deploy Azure OpenAI models

Two LLM models are deployed to the newly created Azure Cognitive Service instance: 

- A chat completion model. In the sample we're deploying `gpt-35-turbo`. This can be replaced with other models providing a chat completion interface like `gpt-4`.
- A text embedding model. In the sample we're deploying `text-embedding-ada-002`. Any other text embedding model can be deployed as well.

In [ ]:
# Chat Completion Model GPT-3.5-turbo
$AOAI_GPT35_DEPLOYMENT = "gpt-35-turbo"
$modelName = "gpt-35-turbo"
$modelVersion = "1106"
$modelFormat = "OpenAI"
$scaleType = "Standard"

az cognitiveservices account deployment create `
   --resource-group $resourceGroup `
   --name $csOpenAIName `
   --deployment-name $AOAI_GPT35_DEPLOYMENT `
   --model-name $modelName `
   --model-version $modelVersion `
   --model-format $modelFormat `
   --sku-name $scaleType `
   --sku-capacity 1

In [ ]:
# Chat Completion Model GPT-4
$AOAI_GPT4_DEPLOYMENT = "gpt-4"
$modelName = "gpt-4"
$modelVersion = "1106-Preview"
$modelFormat = "OpenAI"
$scaleType = "Standard"

az cognitiveservices account deployment create `
   --resource-group $resourceGroup `
   --name $csOpenAIName `
   --deployment-name $AOAI_GPT4_DEPLOYMENT `
   --model-name $modelName `
   --model-version $modelVersion `
   --model-format $modelFormat `
   --sku-name $scaleType `
   --sku-capacity 1

In [ ]:
# Text Embedding Model
$AOAI_EMBEDDING_DEPLOYMENT = "text-embedding-ada-002"
$modelName = "text-embedding-ada-002"
$modelVersion = "2"
$scaleType = "Standard"

az cognitiveservices account deployment create `
   --resource-group $resourceGroup `
   --name $csOpenAIName `
   --deployment-name $AOAI_EMBEDDING_DEPLOYMENT `
   --model-name $modelName `
   --model-version $modelVersion `
   --model-format $modelFormat `
   --sku-name $scaleType `
   --sku-capacity 1

In [ ]:
# GPT-4 Vision Model - GPT-4 Turbo with Vision is the version of GPT-4 that accepts image inputs. It is available as the vision-preview model of gpt-4. The vision-preview model is available in only certain regions. For more information, see
# https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#standard-deployment-model-availability
$AOAI_GPT4VISION_DEPLOYMENT = "gpt-4-vision"
$modelName = "gpt-4"
$modelVersion = "vision-preview"
$scaleType = "Standard"

az cognitiveservices account deployment create `
   --resource-group $resourceGroup `
   --name $csOpenAIName `
   --deployment-name $AOAI_GPT4VISION_DEPLOYMENT `
   --model-name $modelName `
   --model-version $modelVersion `
   --model-format $modelFormat `
   --sku-name $scaleType `
   --sku-capacity 1

Expected output: 

 `JSON string describing the newly deployed models`

## Step 6:   Create Azure Cognitive Search

Azure Cognitive Search is deployed to use its [vector DB functionalities](https://learn.microsoft.com/en-us/azure/search/vector-search-overview). Just like with Azure OpenAI Cognitive Service, the `endpoint` and `API key` of the newly created instance are retrieved for later storage in the `application.env` file.

In [ ]:
$csSearchName = "aisearch$random"
$csSearchSku = "standard"

az search service create `
    --name $csSearchName `
    --resource-group $resourceGroup `
    --location $location `
    --sku $csSearchSku

$csSearchEndpoint = "https://$csSearchName.search.windows.net"

$csSearchApiKey = ( `
    az search admin-key show `
        --resource-group $resourceGroup `
        --service-name $csSearchName `
        --query primaryKey `
        --output tsv `
)

Expected output: 

 `JSON string describing the newly created cognitive search resource`

## Step 7:   Create Azure Storage account

Azure Storage is deployed to store data that can be used to generate indexes in Azure Cognitive Search.

In [ ]:
$stgName = "aistg$random"
$stgSku = "Standard_LRS"

az storage account create `
    --name $stgName `
    --resource-group $resourceGroup `
    --location $location `
    --sku $stgSku `
    --kind StorageV2 `
    --https-only true `
    --access-tier Hot

$stgConnectionString = ( `
    az storage account show-connection-string `
        --name $stgName `
        --resource-group $resourceGroup `
        --query connectionString `
        --output tsv `
)

Expected output: 

`JSON string describing the newly created storage account`

## Step 8: Set environment variables & create application.env file

In [13]:
# Set environment variables

$ENV:AOAI_ENDPOINT = $csOpenAIEndpoint
$ENV:AOAI_APIKEY = $csOpenAIApiKey

#Azure Open AI GPT 3.5
$ENV:AOAI_GPT35_DEPLOYMENT = $AOAI_GPT35_DEPLOYMENT

#Azure Open AI Embedding - text-embedding-ada-002
$ENV:AOAI_EMBEDDING_DEPLOYMENT = $AOAI_EMBEDDING_DEPLOYMENT

#Azure Open AI GPT 4
$ENV:AOAI_GPT4_DEPLOYMENT = $AOAI_GPT4_DEPLOYMENT

#Azure Open AI GPT 4 Vision
$ENV:AOAI_VISION_ENDPOINT = $csOpenAIEndpoint
$ENV:AOAI_VISION_APIKEY = $csOpenAIApiKey
$ENV:AOAI_GPT4VISION_DEPLOYMENT = $AOAI_GPT4VISION_DEPLOYMENT

# Azure Search
$ENV:SEARCH_ENDPOINT = "https://$csSearchEndpoint/"
$ENV:SEARCH_APIKEY = $csSearchApiKey

$ENV:STORAGE_CONNECTIONSTRING = $stgConnectionString
$ENV:ASSET_FOLDER = "../../../../assets"

Write-Host "Environment variables set!"

$configurationFile = "../.env"

function Set-ConfigurationFileVariable($configurationFile, $variableName, $variableValue) {
    if (Select-String -Path $configurationFile -Pattern $variableName) {
        (Get-Content $configurationFile) | Foreach-Object {
            $_ -replace "$variableName = .*", "$variableName=$variableValue"
        } | Set-Content $configurationFile
    } else {
        Add-Content -Path $configurationFile -value "$variableName=$variableValue"
    }
}

Set-ConfigurationFileVariable $configurationFile "AOAI_ENDPOINT" $csOpenAIEndpoint
Set-ConfigurationFileVariable $configurationFile "AOAI_APIKEY" $csOpenAIApiKey
Set-ConfigurationFileVariable $configurationFile "AOAI_GPT35_DEPLOYMENT" $AOAI_GPT35_DEPLOYMENT
Set-ConfigurationFileVariable $configurationFile "AOAI_EMBEDDING_DEPLOYMENT" $AOAI_EMBEDDING_DEPLOYMENT
Set-ConfigurationFileVariable $configurationFile "AOAI_GPT4_DEPLOYMENT" $AOAI_GPT4_DEPLOYMENT
Set-ConfigurationFileVariable $configurationFile "AOAI_VISION_ENDPOINT" $csOpenAIEndpoint
Set-ConfigurationFileVariable $configurationFile "AOAI_VISION_APIKEY" $csOpenAIApiKey
Set-ConfigurationFileVariable $configurationFile "AOAI_GPT4VISION_DEPLOYMENT" $AOAI_GPT4VISION_DEPLOYMENT
Set-ConfigurationFileVariable $configurationFile "SEARCH_ENDPOINT" $csSearchEndpoint
Set-ConfigurationFileVariable $configurationFile "SEARCH_APIKEY" $csSearchApiKey
Set-ConfigurationFileVariable $configurationFile "STORAGE_CONNECTIONSTRING" $stgConnectionString
Set-ConfigurationFileVariable $configurationFile "ASSET_FOLDER" "../../../../assets"


Write-Host "Configuration file created at: $configurationFile"

Environment variables set!
Configuration file created at: ../.env


Expected output:

```
Environment variables set!
Configuration file created at: xxxxxxxxxxxx
```

# 3.3 Basic Chat

In this notebook, we'll explore basic prompt engineering techniques and recommendations that will help us elicit responses from Azure OpenAI Models

### Visual Studio Code

If you are running these steps below in Visual Studio Code make sure you switch your kernal to Python


In [1]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
%pip install openai --user
%pip install panel --user

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip show openai

Name: openai
Version: 1.25.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: C:\Users\joluedem\AppData\Roaming\Python\Python312\site-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from openai import AzureOpenAI

print(os.getenv("AOAI_ENDPOINT"))
print(os.getenv("AOAI_APIKEY"))

client = AzureOpenAI(
  azure_endpoint = os.getenv("AOAI_ENDPOINT"),
  api_key = os.getenv("AOAI_APIKEY"),  
  api_version = "2024-02-15-preview"
)

chatgpt_model_name = os.getenv("AOAI_GPT4_DEPLOYMENT")
print(chatgpt_model_name)

https://westus.api.cognitive.microsoft.com/
2b6f1904286348cf8936389c0d1df7cc
gpt-4


Chat models take a series of messages as input, and return a model-generated message as output. The main input is the messages parameter. Messages must be an array of message objects, where each object has a role (either "system", "user", or "assistant") and content (the content of the message).

In [4]:
# A sample API call for chat completions looks as follows:
# Messages must be an array of message objects, where each object has a role (either "system", "user", or "assistant") and content (the content of the message).
# For more info: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference#chat-completions
import openai
try:
   
    response = client.chat.completions.create(
    model=chatgpt_model_name,
    messages=[
        {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
        {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    ],
    temperature=0,
    max_tokens=800
    )

    print(response.choices[0].message.content)

 
except openai.APIError as e:
    # Handle API error here, e.g. retry or log
    print(f"OpenAI API returned an API Error: {e}")

except openai.AuthenticationError as e:
    # Handle Authentication error here, e.g. invalid API key
    print(f"OpenAI API returned an Authentication Error: {e}")

except openai.APIConnectionError as e:
    # Handle connection error here
    print(f"Failed to connect to OpenAI API: {e}")

except openai.RateLimitError as e:
    # Handle rate limit error
    print(f"OpenAI API request exceeded rate limit: {e}")

except openai.APITimeoutError as e:
    # Handle request timeout
    print(f"Request timed out: {e}")
    
except:
    # Handles all other exceptions
    print("An exception has occured.")

In the land of code, where logic intertwines,
A concept recurs, through the flow of time.
Recursion, a dance, a step so divine,
A function that calls itself, a self-reflected rhyme.

It whispers a question, so simple and pure,
"Shall I invoke myself, once more, once more?"
With each call, a layer, a nesting embrace,
Till the base case is met, the recursive grace.

A mirror within a mirror, depth without end,
Until conditions are met, and it ascends.
Back through each layer, unwinding the stack,
Returning the answers, from the recursive track.

Like Russian dolls, nested one in the next,
Each call is contained, in a recursive text.
But beware the infinite, the loop without cease,
For without a base case, the recursion won't release.

So here is the secret, the heart of the spell,
A base case to break the infinite well.
A condition to stop, to halt the free fall,
And give recursion its meaning, its purpose, its all.

Thus, in the realm of algorithms and lore,
Recursion stands tall, an in

## C# Example:  Setup Parameters

In [14]:
#r "nuget: DotNetEnv, 2.5.0"
#r "nuget: System.Text.Json, 7.0.3"
#r "nuget: Newtonsoft.Json, 13.0.1"
using DotNetEnv;

using System.Net;
using System.Net.Http;
using System.Text.Json.Nodes;
using System.Text.Json;

static string _configurationFile = @"../.env";
Env.Load(_configurationFile);

string apiBase = Environment.GetEnvironmentVariable("AOAI_ENDPOINT"); 
string apiKey = Environment.GetEnvironmentVariable("AOAI_APIKEY"); 
string deploymentName = Environment.GetEnvironmentVariable("AOAI_GPT4_DEPLOYMENT"); 
string apiVersion = "2023-07-01-preview";

Installed Packages DotNetEnv, 2.5.0 Newtonsoft.Json, 13.0.1 System.Text.Json, 7.0.3

Expected output
```
Installed Packages
    DotNetEnv, 2.5.0
    Newtonsoft.Json, 13.0.1
    System.Text.Json, 7.0.3
```

In [15]:
Console.WriteLine(apiBase);
// Console.WriteLine(apiKey);
Console.WriteLine(deploymentName);


https://westus.api.cognitive.microsoft.com/
gpt-4


## Create completions for chat messages with GPT models

The code cell is using an instance of `HttpClient` to call the REST API of the deployed Azure OpenAI instance.

In [16]:
var requestPayload = new JsonObject
{
    { "messages", new JsonArray
        {
            new JsonObject
            {
                { "role", "system" },
                { "content", "You are an AI assistance who You extract intention from provided text. You always answer with intention:" }
                
            },
            new JsonObject
            {
                { "role", "user" },
                { "content", "I'm not receiving calls on my Samsung Galaxy S22. Can you help?" }
            }
        }
    },
    { "max_tokens", 200 },
    { "temperature", 0.7 },
    { "frequency_penalty", 0 },
    { "presence_penalty", 0 },
    { "top_p", 0.95 },
    { "stop", null }
};

string payload = JsonSerializer.Serialize(requestPayload, new JsonSerializerOptions
{
    WriteIndented = true // Optional: to make the JSON string more readable
});

        
string endpoint = $"{apiBase}openai/deployments/{deploymentName}/chat/completions?api-version={apiVersion}";

using (HttpClient httpClient = new HttpClient())
{
    httpClient.BaseAddress = new Uri(endpoint);
    httpClient.DefaultRequestHeaders.Add("api-key",apiKey);
    httpClient.DefaultRequestHeaders.Accept.Add(new System.Net.Http.Headers.MediaTypeWithQualityHeaderValue("application/json"));

    var stringContent = new StringContent(payload, Encoding.UTF8, "application/json");

    var response = await httpClient.PostAsync(endpoint, stringContent);

    if (response.IsSuccessStatusCode)
    {
        using (var responseStream = await response.Content.ReadAsStreamAsync())
        {
            // Parse the JSON response using JsonDocument
            using (var jsonDoc = await JsonDocument.ParseAsync(responseStream))
            {
                // Access the message content dynamically
                var root = jsonDoc.RootElement;
                var messageContent = root.GetProperty("choices")[0].GetProperty("message").GetProperty("content").GetString();

                // Output the message content
                Console.WriteLine("Output: " + messageContent);
            }
        }
    }
    else
    {
        Console.WriteLine($"Error: {response}");
    }
}

Output: It seems you are experiencing issues with receiving calls on your Samsung Galaxy S22 and you're seeking assistance to resolve this problem.


Expected output:

```
Output: Intention: Requesting technical assistance with phone call issue on Samsung Galaxy S22.
```

# 3.4 Tokenization

In this notebook, we'll explore basic concepts behind tokenization, how to use the Microsoft.ML.Tokenizers library to tokenize text and get information about token counts

https://github.com/Azure-Samples/openai-dotnet-samples/blob/main/tokenization.ipynb

### Install Microsoft.ML.Tokenizers

In [17]:
#r "nuget:Microsoft.ML.Tokenizers"

Installed Packages Microsoft.ML.Tokenizers, 0.21.1

Installed Packages
 - Microsoft.ML.Tokenizers, 0.21.1

### Add using statements

In [18]:
using Microsoft.ML.Tokenizers;

### Download and define vocab resources

Download the following files and place them in the root directory. These vocabulary files are what are used to encode the text into tokens.

- [GPT Vocabulary Files](https://huggingface.co/gpt2/tree/main)
    - vocab.json
    - merges.txt

In [19]:
var vocabFilePath = @"../notebooks/assets/vocab.json";
var mergeFilePath = @"../notebooks/assets/merges.txt";

### Initialize Tokenizer

In [20]:
var tokenizer = new Tokenizer(new Bpe(vocabFilePath, mergeFilePath));

### Encode text into tokens

In [21]:
var input = "the brown fox jumped over the lazy dog!";
var tokenizerEncodedResult = tokenizer.Encode(input);
tokenizerEncodedResult

Microsoft.ML.Tokenizers.TokenizerResult OriginalString the brown fox jumped over the lazy dog! NormalizedString the brown fox jumped over the lazy dog! OffsetsMappedToOriginalString True Ids [ 1169, 33282, 12792, 73, 27073, 2502, 1169, 75, 12582, 9703, 0 ] Tokens [ the, brown, fox, j, umped, over, the, l, azy, dog, ! ] Offsets index value 0 (0, 3) Item1 0 Item2 3 1 (4, 9) Item1 4 Item2 9 2 (10, 13) Item1 10 Item2 13 3 (14, 15) Item1 14 Item2 15 4 (15, 20) Item1 15 Item2 20 5 (21, 25) Item1 21 Item2 25 6 (26, 29) Item1 26 Item2 29 7 (30, 31) Item1 30 Item2 31 8 (31, 34) Item1 31 Item2 34 9 (35, 38) Item1 35 Item2 38 10 (38, 39) Item1 38 Item2 39

### Get token count

In [22]:
tokenizerEncodedResult.Tokens.Count()

11

# 3.5 Prompts & Completions

In this section, we'll explore small prompt engineering techniques, prompt construction, and recommendations that will help us elicit responses from the models that are better suited to our needs. In addition, the techniques in this section will teach you strategies for increasing the accuracy and grounding of responses you generate with a Large Language Model (LLM).

In [5]:
%pip install openai
%pip install panel 
%pip install semantic_kernel
%pip install common

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 1.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 60.6/60.6 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to 

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python312\\Scripts\\dotenv.exe' -> 'c:\\Python312\\Scripts\\dotenv.exe.deleteme'



In [24]:
%pip install common 
%pip install semantic_kernel

Note: you may need to restart the kernel to use updated packages.
  Using cached semantic_kernel-0.9.6b1-py3-none-any.whl.metadata (5.2 kB)
  Using cached aiohttp-3.9.5-cp312-cp312-win_amd64.whl.metadata (7.7 kB)
  Using cached grpcio-1.63.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached motor-3.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached openapi_core-0.19.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached prance-23.6.21.0-py3-none-any.whl.metadata (13 kB)
  Using cached pybars4-0.9.13-py3-none-any.whl
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp312-cp312-win_amd64.whl.metadata (4.3 kB)
  Using cached yarl-1.9.4-cp312-cp312-win_amd64.whl.metadata (32 kB)
  Using cached pymongo-4.7.1-cp312-cp312-win_amd64.whl.metadata (22 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl.metadata (9.6 kB)
  Using cached jsonschema_path-0.3.2-py3-none

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python312\\Scripts\\chardetect.exe' -> 'c:\\Python312\\Scripts\\chardetect.exe.deleteme'



In [ ]:
import os
from openai import AzureOpenAI

print(os.getenv("AOAI_ENDPOINT"))
print(os.getenv("AOAI_APIKEY"))

client = AzureOpenAI(
  azure_endpoint = os.getenv("AOAI_ENDPOINT"),
  api_key = os.getenv("AOAI_APIKEY"),  
  api_version = "2024-02-15-preview"
)

In [7]:
response = client.chat.completions.create(
    model="gpt-35-turbo", # model = "deployment_name".
    messages=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "Who were the founders of Microsoft?"}
    ]
)

#print(response)
print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)

{
  "id": "chatcmpl-9KEFFb3yPuNiNv4T834xmBQydtjwG",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Microsoft was founded by Bill Gates and Paul Allen on April 4, 1975.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1714608345,
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "system_fingerprint": "fp_2f57f81c11",
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 29,
    "total_tokens": 47
  },
  "prompt_filter_r

# 3.6 Techniques

In this section, we'll delve into techniques designed to enhance the accuracy and coherence of responses generated by a Large Language Model (LLM). These strategies will equip you with the skills to produce more precise and well-grounded outputs.

# Formating the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot - which refers to providing no examples
* One-Shot.
* Few-Shots - The term few-shot refers to providing a few of examples to help the model learn what it needs to do

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.


In [8]:
#Functio to call the model.
def return_OAIResponse(user_message, context):

#As we can see, we’re adding the user’s question at the end of the prompt with the user role, so the model understands that this is a user request and not an instruction on how it should work.
    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    # print(newcontext)

    response = client.chat.completions.create(
        model="gpt-35-turbo", # model = "deployment_name".
        messages=newcontext,
        temperature=0,
        max_tokens=800
    )

    # print(response)
    print(response.model_dump_json(indent=2))

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [9]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

{
  "id": "chatcmpl-9KEGEDL3Yv7tAJlE0B72vjgNoIseD",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Sebastian Vettel won the F1 2010 championship.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1714608406,
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "system_fingerprint": "fp_2f57f81c11",
  "usage": {
    "completion_tokens": 13,
    "prompt_tokens": 30,
    "total_tokens": 43
  },
  "prompt_filter_results": [
    {
     

In [10]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

{
  "id": "chatcmpl-9KEHEAxhd0mfyzhldufInTPeLnm2q",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Driver: Sebastian Vettel\nTeam: Red Bull Racing",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1714608468,
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "system_fingerprint": "fp_2f57f81c11",
  "usage": {
    "completion_tokens": 11,
    "prompt_tokens": 54,
    "total_tokens": 65
  },
  "prompt_filter_results": [
    {
    

Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [11]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Bettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

{
  "id": "chatcmpl-9KEICZ7FwuOgjw4kt96uzL8SXtHRU",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The 2006 F1 championship was won by Fernando Alonso, driving for the Renault F1 Team.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1714608528,
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "system_fingerprint": "fp_2f57f81c11",
  "usage": {
    "completion_tokens": 21,
    "prompt_tokens": 80,
    "total_tokens": 101
  },


In [12]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

{
  "id": "chatcmpl-9KEJBxIu76OtYHa8JmwbJMCJ5UPPo",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The 2019 F1 championship was won by Lewis Hamilton, driving for Mercedes.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1714608589,
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "system_fingerprint": "fp_2f57f81c11",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 80,
    "total_tokens": 97
  },
  "prompt_fil

We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [13]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Bettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

{
  "id": "chatcmpl-9KEKRTY37pjKR87mjB0vNNU3meut5",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Lewis Hamilton won the F1 2019 championship.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1714608667,
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "system_fingerprint": "fp_2f57f81c11",
  "usage": {
    "completion_tokens": 11,
    "prompt_tokens": 105,
    "total_tokens": 116
  },
  "prompt_filter_results": [
    {
     

We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [14]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answew the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

{
  "id": "chatcmpl-9KELQx4tRDbztTH0Qdbldlf9sUfNi",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Driver: Lewis Hamilton\nTeam: Mercedes\nPoints: 413",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1714608728,
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "system_fingerprint": "fp_2f57f81c11",
  "usage": {
    "completion_tokens": 13,
    "prompt_tokens": 77,
    "total_tokens": 90
  },
  "prompt_filter_results": [
    {


In [15]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Bettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

{
  "id": "chatcmpl-9KEMQrod6S9iBp2WZY9OfkfXkbpN7",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Driver: Fernando Alonso.\nTeam: Renault.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1714608790,
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "system_fingerprint": "fp_2f57f81c11",
  "usage": {
    "completion_tokens": 9,
    "prompt_tokens": 77,
    "total_tokens": 86
  },
  "prompt_filter_results": [
    {
      "promp

### Few Shots for classification.

In [16]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Setiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

{
  "id": "chatcmpl-9KENPjvsQ8TpB7rJwglWAlbD6eFzW",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Sentiment: Neutral",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1714608851,
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "system_fingerprint": "fp_2f57f81c11",
  "usage": {
    "completion_tokens": 4,
    "prompt_tokens": 134,
    "total_tokens": 138
  },
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "

In [17]:
context_user=[
        {"role": "system", "content": "You are an OrderBot in a fastfood restaurant."},
        {"role": "user", "content": "I have only 10 dollars, what can I order?"},
        {"role": "assistant", "content": "We have the fast menu for 7 dollars."},
        {"role": "user", "content": "Perfect! Give me one! "}
]
print(return_OAIResponse("", context_user))

{
  "id": "chatcmpl-9KEOO0tMizQcV9xaq2Q0YRT27qZW1",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "How can I help you?",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1714608912,
  "model": "gpt-35-turbo",
  "object": "chat.completion",
  "system_fingerprint": "fp_2f57f81c11",
  "usage": {
    "completion_tokens": 6,
    "prompt_tokens": 66,
    "total_tokens": 72
  },
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "c

## Content Generation

In this section, we'll explore how to use LLMs to do content generation

In [3]:
import common
import semantic_kernel

# Get a configured model
client = common.get_openai_client(api_key=common.api_KEY,
        api_version=common.api_version,
        azure_endpoint=common.api_URI)

In [4]:
def mock_get_car() -> dict:
    return {
        "make": "Ford",
        "model": "Explorer",
        "base": "XLT",
        "color": "red",
        "year": 2019,
        "condition": "good",
        "mileage": 32000,	
        "price": 25000
    }

def get_car_description(car: dict) -> str:
    return f'{car["year"]} {car["make"]} {car["model"]} {car["base"]} {car["color"]} with {car["mileage"]} miles in {car["condition"]} condition for ${car["price"]}.'

car = mock_get_car()
car_description = get_car_description(car)

# Create a semantic kernel inline function
sales_desc_generation_template = "Create a one paragraph sales description that includes the price for a {{input}}"
template = common.render_template(sales_desc_generation_template, input=car_description)

# Execute the SK function
print(common.Call_OpenAI(client,common.gpt_api_deployment,template,max_tokens=500))

Introducing the 2019 Ford Explorer XLT in a stunning red color with only 32000 miles on the odometer. This vehicle is in excellent condition and is ready to take you on your next adventure. Priced at just $25000, this Explorer offers a spacious interior, powerful V6 engine, and a smooth ride. Don't miss out on this great deal! Contact us today to schedule a test drive.


## Classification

In this section, we'll explore how to use LLMs to do classification

In [5]:
import common

# Get a configured model
client = common.get_openai_client(api_key=common.api_KEY,
        api_version="2024-02-15-preview",
        azure_endpoint=common.api_URI)

print(common.api_URI)

https://westus.api.cognitive.microsoft.com/


In [6]:
prompt_template = """For the following list of animals:

- Dog
- Cat
- Elephant
- Dolphin
- Shark
- Whale
- Snake

Can you classify and list by animal type?
"""

result = common.Call_OpenAI(client,common.gpt_api_deployment,prompt_template)

print(result)

Mammals:
- Dog
- Cat
- Elephant
- Dolphin
- Whale

Fish/Reptiles:
- Shark
- Snake


## Recommendations

In this section, we'll explore how to use LLMs to do recommendations

In [7]:
import common

# Get a configured model
client = common.get_openai_client(api_key=common.api_KEY,
        api_version=common.api_version,
        azure_endpoint=common.api_URI)

In [8]:
def mock_get_restaurant_list(cityCode) -> list[str]:
    if (cityCode == "MIA"):
        return [
            "Joe's Stone Crab",
            "Versailles",
            "Hillstone",
            "Casa Tua",	
            "Cecconi's",
            "Yardbird Southern Table & Bar",
        ]
    return []

target_text=""
for restaurant in mock_get_restaurant_list("MIA"):
    target_text += f"{restaurant}\n"

print(target_text)

recommendation_template = 'List two top restaurants:\n{{input}}\nOut in JSON format.'
print(recommendation_template)
rendered_template = common.render_template(recommendation_template, input=target_text)
print(rendered_template)

print(common.Call_OpenAI(client,common.gpt_api_deployment,rendered_template,max_tokens=200))

Joe's Stone Crab
Versailles
Hillstone
Casa Tua
Cecconi's
Yardbird Southern Table & Bar

List two top restaurants:
{{input}}
Out in JSON format.
List two top restaurants:
Joe's Stone Crab
Versailles
Hillstone
Casa Tua
Cecconi's
Yardbird Southern Table & Bar

Out in JSON format.
{
  "restaurants": [
    {
      "name": "Joe's Stone Crab",
      "type": "Seafood",
      "location": "11 Washington Ave, Miami Beach, FL 33139"
    },
    {
      "name": "Versailles",
      "type": "Cuban",
      "location": "3555 SW 8th St, Miami, FL 33135"
    },
    {
      "name": "Hillstone",
      "type": "American",
      "location": "201 Miracle Mile, Coral Gables, FL 33134"
    },
    {
      "name": "Casa Tua",
      "type": "Italian",
      "location": "1700 James Ave, Miami Beach, FL 33139"
    },
    {
      "name": "Cecconi's",
      "type": "Italian",
      "location": "4385 Collins Ave, Miami Beach, FL 33140"



## Translation with Semantic Kernal

In this section, we'll explore how to use LLMs to do recommendations

In [9]:
%pip install semantic_kernel

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip show semantic_kernel

Name: semantic-kernel
Version: 0.9.6b1
Summary: Semantic Kernel Python SDK
Home-page: 
Author: Microsoft
Author-email: SK-Support@microsoft.com
License: 
Location: C:\Users\joluedem\AppData\Roaming\Python\Python311\site-packages
Requires: aiohttp, defusedxml, grpcio, jinja2, motor, nest-asyncio, numpy, openai, openapi_core, prance, pybars4, pydantic, python-dotenv, regex, scipy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [11]:
import common

# Get a configured model
client = common.get_openai_client(api_key=common.api_KEY,
        api_version=common.api_version,
        azure_endpoint=common.api_URI)

print(common.api_version)
print(common.api_URI)

2023-07-01-preview
https://westus.api.cognitive.microsoft.com/


In [5]:
import semantic_kernel as sk

kernel = sk.Kernel()

In [ ]:
import common

# Get a configured Semantic Kernel
# Note all other demos except this one use the OpenAI SDK
kernel = common.get_kernel()

In [ ]:
languageCodes = {
    "en": "English",
    "es": "Spanish",
}

def mock_get_extract_language(languageCode) -> str:
    if (languageCode == "en"):
        return '''Azure Container Apps is a fully managed environment that enables you to run microservices and containerized applications on a serverless platform. Common uses of Azure Container Apps include:
Deploying API endpoints
Hosting background processing applications
Handling event-driven processing
Running microservices'''
    return ""


source_language = languageCodes["en"]  # English
target_language = languageCodes["es"]  # Spanish
target_text = mock_get_extract_language("en")

translation_template = "'Translate the text from {{$Source}} to {{$Target}}.\nText:\n\n{{$input}}'"
translationFunc = kernel.create_semantic_function(translation_template,max_tokens=100,temperature=0.3)
context = kernel.create_new_context()
context['Source'] = source_language
context['Target'] = target_language
context['input'] = target_text

bot_answer = await translationFunc.invoke_async(context=context)
print(bot_answer)

## Sentiment

In this section, we'll explore how to use LLMs to do sentiment analysis

In [6]:
import common

# Get a configured model
client = common.get_openai_client(api_key=common.api_KEY,
        api_version=common.api_version,
        azure_endpoint=common.api_URI)

In [7]:
import json

reviews = [
    """Mark S.: \"This smartphone has exceeded my expectations in every way. The camera quality is exceptional, capturing vivid details and colors. The battery life is impressive, easily lasting a full day with heavy usage. The sleek design and intuitive interface make it a pleasure to use. I'm a satisfied customer and would recommend it to anyone!\"""",
    """Maya L.: \"I've been using this phone for a few weeks now, and it's a game-changer. The processing speed is fantastic, and I've experienced no lag even with multiple apps running. The display is vibrant, making videos and games look stunning. The fingerprint sensor is quick and accurate. Overall, a solid choice for anyone looking for a reliable and feature-packed smartphone.\"""",
    """David W.: \"What sets this smartphone apart is its seamless integration with other devices. The ecosystem it creates is truly impressive, making my daily tasks more efficient. The build quality is robust, and the phone feels great in hand. The software updates have been regular, showing a commitment to keeping the device up to date. I'm very happy with my purchase.\"""",
    """Jennifer P.: \"The smartphone is decent overall. The camera takes good photos, and the battery life is acceptable. However, I expected a bit more from the performance – there's a slight lag at times, especially when running resource-intensive apps. The design is standard, nothing particularly eye-catching. It's a reliable phone, but not a standout in the market.\"""",
    """Ryan M.: \"This smartphone is a complete disappointment. The camera quality is subpar, producing grainy and washed-out photos. The battery drains rapidly, and even with minimal usage, it struggles to last half a day. The software is buggy, with frequent crashes and unresponsive touch screen. Save yourself the trouble and look elsewhere. This phone is not worth the money.\"""",    
]

# Create a semantic kernel inline function
sentiment_template = """From 1-10, 10 being a an excellent sentiment. What is the sentiment for: {{input}}?

Output format: {\"sentiment\": 5}

Output in JSON format only."""

# Execute the SK function
total_score = 0
for review in reviews:
    template = common.render_template(sentiment_template, input=review)
    data = common.Call_OpenAI(client,common.gpt_api_deployment,template)
    score = json.loads(data)['sentiment']
    print(f'Review: {review[0:60]}, Score: {score}')
    total_score += score

print("Average sentiment score: {}".format(total_score/len(reviews)))

Review: Mark S.: "This smartphone has exceeded my expectations in ev, Score: 10
Review: Maya L.: "I've been using this phone for a few weeks now, an, Score: 9
Review: David W.: "What sets this smartphone apart is its seamless i, Score: 9
Review: Jennifer P.: "The smartphone is decent overall. The camera t, Score: 5
Review: Ryan M.: "This smartphone is a complete disappointment. The , Score: 1
Average sentiment score: 6.8


## Analysis

In this section, we'll explore how to use LLMs to do analysis

In [8]:
import common

# Get a configured model
print(common.api_version)
print(common.gpt_api_deployment)

client = common.get_openai_client(api_key=common.api_KEY,
        api_version=common.api_version,
        azure_endpoint=common.api_URI)

2023-07-01-preview
gpt-35-turbo


In [9]:
def mock_get_contract(id: str) -> str:
    if id == "LEASE_AGREEMENT":
        return '''RESIDENTIAL LEASE AGREEMENT
RENT. The Tenant shall pay to Landlord the sum of $1,500 per month (hereinafter referred to as "Rent") for the duration of the Term of the Lease. The Rent shall be payable on or before every day of the month (hereinafter referred to as the "Due Date"), notwithstanding that the said date falls on a weekend or holiday.
A. Late Rent. If Rent is not paid within days of the Due Date, the Rent shall be considered past due, and a late fee of a $50 or 5% of the Rent past due shall be applied for every day Rent is late or O occurrence Rent is late.
B. Returned Checks. In the event that a check intended as payment for Rent is dishonored for whatever reason, the same shall be considered as Late Rent with the late fee being payable on the same.'''
    return ""

In [10]:
# get a contract
contract = mock_get_contract("LEASE_AGREEMENT")

# print(contract)
# Create a semantic kernel inline function
analysis_template = """System:
You are an agent that can help summarize and analyze contracts for risk. Be professional and courteous.

User:
For the following text, summarize and list risks. \n{{input}}

Output format:
Summary: 
""

Risks: 
-||-

"""

In [11]:
# Execute the function
template = common.render_template(analysis_template, input=contract)
print(template)
print(common.Call_OpenAI(client,common.gpt_api_deployment,template,max_tokens=300))

System:
You are an agent that can help summarize and analyze contracts for risk. Be professional and courteous.

User:
For the following text, summarize and list risks. 
RESIDENTIAL LEASE AGREEMENT
RENT. The Tenant shall pay to Landlord the sum of $1,500 per month (hereinafter referred to as "Rent") for the duration of the Term of the Lease. The Rent shall be payable on or before every day of the month (hereinafter referred to as the "Due Date"), notwithstanding that the said date falls on a weekend or holiday.
A. Late Rent. If Rent is not paid within days of the Due Date, the Rent shall be considered past due, and a late fee of a $50 or 5% of the Rent past due shall be applied for every day Rent is late or O occurrence Rent is late.
B. Returned Checks. In the event that a check intended as payment for Rent is dishonored for whatever reason, the same shall be considered as Late Rent with the late fee being payable on the same.

Output format:
Summary: 
""

Risks: 
-||-

Summary:
The re

## Scoring

In this section, we'll explore how to use LLMs to do scoring

In [12]:
import common

# Get a configured model
client = common.get_openai_client(api_key=common.api_KEY,
        api_version=common.api_version,
        azure_endpoint=common.api_URI)

In [13]:
songs = [
    ['La Canción de los Gatos', """En el jardín, jugando van,
Pequeños gatos, con gran afán.
Pelos suaves, y orejas puntiagudas,
Saltan y juegan, ¡qué travesuras!

¡Miau, miau, los gatos juegan así!
En la luz del sol, o bajo la luna, sí.
¡Miau, miau, saltan con destreza,
Los gatos son la pura belleza!

Con bigotes finos, y ojos brillantes,
Exploran rincones, son tan elegantes.
Persiguen mariposas, atrapan ratones,
Los gatos son reyes de los callejones.

¡Miau, miau, los gatos juegan así!
En la luz del sol, o bajo la luna, sí.
¡Miau, miau, saltan con destreza,
Los gatos son la pura belleza!

Descansan en tejados, bajo el cielo estrellado,
Ronroneando suavemente, a veces hasta dormitando.
Cada gato, con su propia personalidad,
¡Son pequeños amigos llenos de vitalidad!

¡Miau, miau, los gatos juegan así!
En la luz del sol, o bajo la luna, sí.
¡Miau, miau, saltan con destreza,
Los gatos son la pura belleza!

Y así termina la canción de los gatos,
Con sus travesuras y sus saltos.
Gatitos felices, en su propio rincón,
¡Que la alegría de los gatos siga en tu corazón!""", "Hard"],
    ['Conquistando el Amor', """En la penumbra de la noche, perdido en el laberinto,
Caminando entre susurros, buscando el amor instinto.
Ojos que brillan como estrellas, en la oscuridad se encuentran,
Corazones en batalla, donde las sombras se entrelazan.

Conquistando el amor, en un juego sin final,
Donde las promesas se tejen, como hilos en el cristal.
Entre suspiros y secretos, dos almas se entrelazan,
En este duelo de pasiones, el amor se abalanza.

En el campo de las emociones, donde la razón se desvanece,
Se libra la batalla, entre la dicha y la tristeza.
Susurros de seducción, en la danza de la pasión,
Labios que pronuncian versos, creando un lazo de unión.

Bajo el cielo estrellado, donde los sueños se conjugan,
Se forja la alianza, que en el corazón se anida.
Palabras como flechas, atraviesan el silencio,
Conquistando el amor, en un eterno encuentro.

Conquistando el amor, en un juego sin final,
Donde las promesas se tejen, como hilos en el cristal.
Entre suspiros y secretos, dos almas se entrelazan,
En este duelo de pasiones, el amor se abalanza.

En el jardín de los sentimientos, donde florece la esperanza,
Se escribe la historia, de una conquista que avanza.
Manos que se buscan, en la penumbra del deseo,
Conquistando el amor, como un fuego que no teme.

Bajo el manto de la Luna, sellando el pacto eterno,
Dos corazones en victoria, en este amor moderno.
Conquistando el amor, como héroes en la trama,
En este cuento sin final, donde el amor se proclama.
""", "Easy"]    
]

for song in songs:
    
    prompt_template = """You are an agent who can help determine how easy it would be for an English speaker to learn to sing a song in Spanish. Easy songs have straightforward vocabulary and grammar and avoid complex sentence structures, metaphors, poetic structures and language, and uncommon words. Songs with familiar or universal themes, such as love, emotions, or everyday activities, can be easier for learners to relate to and understand. It helps when the context of the song is relatable to the listener. 

    Rate the following song lyrics in Spanish from 1-10, with 10 being the hardest, for an English speaker to learn:

    Song Title: \"\"\"
    {{title}}
    \"\"\"

    Lyrics: \"\"\"
    {{input}}
    \"\"\"

    
    Output format:
    { \"score\":-1, \"title\":\"\",\"explanation\": \"\"}

    Provide an explanation in one sentence. Output in JSON format only.
    """

    rendered_template = common.render_template(prompt_template,title=song[0],input=song[1])    
    print(common.Call_OpenAI(client, common.gpt_api_deployment, rendered_template))

{"score":8, "title":"La Canción de los Gatos", "explanation": "The song has complex vocabulary and descriptive language, making it challenging for an English speaker to learn."}
{"score":10, "title":"Conquistando el Amor", "explanation": "The song contains complex vocabulary, poetic language, and metaphors, making it very challenging for an English speaker to learn."}


# Validation

### Score how similar a baseline answer to an actual answer.

In [14]:
import common

# Get a configured model
client = common.get_openai_client(
        api_key=common.api_KEY,
        api_version=common.api_version,
        azure_endpoint=common.api_URI)

### Create a Jinja2 template

In [15]:
prompt_template = """On a scale from 0-10 with 10 being a very good match, how close of a match is the Actual Response to the Baseline Response:

Baseline response:
{{EXPECTED_ANSWER}}

Actual Response:
{{ANSWER}}

Output format:
{
"score":0,
"reason":""
}

Output in JSON format only."""

### Check a good answer

In [16]:
EXPECTED_ANSWER = "The speed of light in a vacuum is approximately 299,792,458 meters per second (m/s). To convert this value to kilometers per second (km/s), we divide by 1,000 since there are 1,000 meters in a kilometer. Therefore, the speed of light in kilometers per second is approximately 299,792.458 km/s."
ANSWER = "The speed of light is approximately 300,000 km/s."
prompt = rendered_template = common.render_template(prompt_template, EXPECTED_ANSWER=EXPECTED_ANSWER, ANSWER=ANSWER)

In [17]:
result = common.Call_OpenAI(client,common.gpt_api_deployment,prompt,temperature=0.0)
print(result)

{
  "score": 5,
  "reason": "The actual response is close in value to the baseline response, but it is not an exact match."
}


### Check a bad answer

In [18]:
EXPECTED_ANSWER = "The company offers paid sick leave, vacation, and paid medical insurance."
ANSWER = "Some of the benefits are paid holidays and 401k matching."
prompt = rendered_template = common.render_template(prompt_template, EXPECTED_ANSWER=EXPECTED_ANSWER, ANSWER=ANSWER)

In [19]:
result = common.Call_OpenAI(client,common.gpt_api_deployment,prompt,temperature=0.0)
print(result)

{
  "score": 3,
  "reason": "The Actual Response mentions some benefits, but it does not specifically mention paid sick leave, vacation, and paid medical insurance as in the Baseline Response."
}


# Intent

In this section, we'll explore how to use LLMs to do recommendations

In [20]:
import common

# Get a configured model
client = common.get_openai_client(api_key=common.api_KEY,
        api_version=common.api_version,
        azure_endpoint=common.api_URI)

In [21]:
intent_template = """System:
You are a travel assistant that can help determine intent. An intent is an action from the list of Defined Intents. An action is defined as a place, location, name, time or date.

Defined Intents:
GetItinerary
Reserve
CancelReservation
CheckReservation
GetReservation
GetWeather
Unknown

User:
What is the intent and entities for the following request:
{{input}}

Output format:
{
  "intent": "intent",
  "entities": ["action"]
}

Do not provide explanations. Output in JSON format only."""

In [22]:
print(common.Call_OpenAI(client,common.gpt_api_deployment,common.render_template(intent_template,input="I want to make a reservation for 2 people at 7pm on Friday at Friday's Restaurant.")))

{
  "intent": "Reserve",
  "entities": ["Friday's Restaurant", "7pm", "Friday", "2 people"]
}


In [23]:
print(common.Call_OpenAI(client,common.gpt_api_deployment,common.render_template(intent_template,input="What is my upcoming travel itinerary?")))

{
  "intent": "GetItinerary",
  "entities": []
}


In [24]:
print(common.Call_OpenAI(client,common.gpt_api_deployment,common.render_template(intent_template,input="What is the weather like in London?")))

{
  "intent": "GetWeather",
  "entities": ["London"]
}


In [25]:
print(common.Call_OpenAI(client,common.gpt_api_deployment,common.render_template(intent_template,input="What is the speed of light?")))

{
  "intent": "Unknown",
  "entities": []
}


# Function Calling

The latest versions of gpt-35-turbo and gpt-4 are fine-tuned to work with functions and are able to both determine when and how a function should be called. 
In this section, we'll explore how to work with LLMs and functions

- Call the chat completions API with your functions and the user’s input
- Use the model’s response to call your API or function
- Call the chat completions API again, including the response from your function to get a final response

In [30]:
import os
from openai import AzureOpenAI
import json

print(os.getenv("AOAI_ENDPOINT"))
print(os.getenv("AOAI_APIKEY"))
print(os.getenv("AOAI_GPT35_DEPLOYMENT"))

client = AzureOpenAI(
  azure_endpoint = os.getenv("AOAI_ENDPOINT"), 
  api_key=os.getenv("AOAI_APIKEY"),  
  api_version="2024-03-01-preview",
  #model=os.getenv("AOAI_GPT35_DEPLOYMENT")
)

# Example function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model=os.getenv("AOAI_GPT35_DEPLOYMENT"),
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model=os.getenv("AOAI_GPT4_DEPLOYMENT"),
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
print(run_conversation())

https://westus.api.cognitive.microsoft.com/
2b6f1904286348cf8936389c0d1df7cc
gpt-35-turbo
ChatCompletion(id='chatcmpl-9KFVWxqqcFGHL3KSZAIhA8hXByhTR', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="As of the latest information I have:\n\n- In San Francisco, the temperature is approximately 72°C. (This seems unusually high and is likely incorrect, please check a reliable local weather service for accurate information)\n- In Tokyo, the temperature is around 10°C.\n- In Paris, the temperature is about 22°C.\n\nPlease note that weather conditions can change rapidly, so for real-time data and to verify the unusually high temperature reported for San Francisco, it's best to consult a reliable weather forecasting service.", role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severi

# 3.7 Embeddings & Vector DBs

In this section, we'll explore Vector DBs, Embeddings and Chunking

In [3]:
from PyPDF2 import PdfReader
# from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv,dotenv_values
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [4]:
FILE_PATH = "G:\\OpenAI-DataPro\\notebooks\\assets\\azure-ai-services-openai.pdf"
print(FILE_PATH)

G:\OpenAI-DataPro\notebooks\assets\azure-ai-services-openai.pdf


In [5]:
# Read the PDF file and return the text
def get_pdf_data(file_path, num_pages = 1):
    reader = PdfReader(file_path)
    full_doc_text = ""
    pages = reader.pages
    num_pages = len(pages) 
    
    try:
        for page in range(num_pages):
            current_page = reader.pages[page]
            text = current_page.extract_text()
            full_doc_text += text
    except:
        print("Error reading file")
    finally:
        return full_doc_text

In [6]:
# Divide the text into chunks of chunk_length 
# [ default is 500] characters
def get_chunks(fulltext:str,chunk_length =500) -> list:
    text = fulltext

    chunks = []
    while len(text) > chunk_length:
        last_period_index = text[:chunk_length].rfind('.')
        if last_period_index == -1:
            last_period_index = chunk_length
        chunks.append(text[:last_period_index])
        text = text[last_period_index+1:]
    chunks.append(text)

    return chunks

In [7]:
filename = FILE_PATH
FILE_PATH.split('/')[-1]

'G:\\OpenAI-DataPro\\notebooks\\assets\\azure-ai-services-openai.pdf'

In [8]:
full_doc_text = get_pdf_data(filename)
print(f'Full doc text length: {len(full_doc_text)}')

Full doc text length: 909339


In [9]:
Lines =get_chunks(full_doc_text,500)
len(Lines)

2162

In [10]:
type(Lines)

list

In [11]:
from openai import AzureOpenAI
import os

load_dotenv()  # make sure to have the .env file in the root directory of the project

client = AzureOpenAI(
  azure_endpoint = os.getenv("AOAI_ENDPOINT"),
  api_key = os.getenv("AOAI_APIKEY"),  
  api_version = "2024-02-15-preview"
)

model: str = os.getenv("AOAI_EMBEDDING_DEPLOYMENT")

In [12]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(text, model=model):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [13]:
counter = 0
input_data = []

In [ ]:
%%time
for line in Lines:
    d = {}
    d['id'] = str(counter)
    d['line'] = line
    d['embedding'] = generate_embeddings(line)
    d['filename'] = FILE_PATH.split('/')[-1]
    counter = counter + 1
    input_data.append(d)

In [15]:
counter

549

In [16]:
len(input_data[0]['embedding'])

1536

In [17]:
# Output embeddings to docVectors.json file
with open("output/docVectors_azure.json", "w") as f:
    json.dump(input_data, f)

# 3.8 SDKs and Orchestration

In this section, we'll explore different SDKs (LangChan, Semantic Kernal) and orchestration methods such as Assistants APIs


### Math Tutor Assistant

This sample demonstrates the following:

- Showcases the foundational concepts of Assistants such as Threads, Messages, Runs, Tools, and lifecycle management.

This sample shows users how to create an Azure OpenAI Assistant named "Math Tutor" using the Azure OpenAI API. The assistant is designed to function as a personal math tutor, capable of answering math questions through code interpretation. The script initiates a conversation with the assistant, guiding it through various mathematical queries and scenarios to showcase its capabilities.

This sample provides developers with a clear demonstration of how to leverage the core concepts of the Assistants API into their projects, highlighting its simplicity and effectiveness in leveraging foundational concepts.

In [1]:
import os

from dotenv import load_dotenv

#load_dotenv()  # make sure to have the .env file in the root directory of the project

api_endpoint = os.getenv("AOAI_VISION_ENDPOINT")
api_key = os.getenv("AOAI_VISION_APIKEY")
api_version = "2024-02-15-preview"
api_deployment_name = os.getenv("AOAI_GPT4VISION_DEPLOYMENT")

should_cleanup: bool = False

## Run this Example
### Load the required libraries

In [2]:
import io
import time
from datetime import datetime
from typing import Iterable

from openai import AzureOpenAI
from openai.types.beta.threads.message_content_image_file import MessageContentImageFile
from openai.types.beta.threads.message_content_text import MessageContentText
from openai.types.beta.threads.messages import MessageFile
from PIL import Image

### Create an Azure OpenAI client

In [4]:
client = AzureOpenAI(
api_key=os.getenv("AOAI_VISION_APIKEY"), 
api_version=os.getenv("AOAI_GPT4VISION_DEPLOYMENT"),
azure_endpoint=os.getenv("AOAI_VISION_ENDPOINT")
)

### Create an Assistant and a Thread

In [ ]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model=api_deployment_name,
)

thread = client.beta.threads.create()

### Format and display the Assistant Messages for text and images

In [ ]:
def read_assistant_file(file_id:str):
    response_content = client.files.content(file_id)
    return response_content.read()

def print_messages(messages: Iterable[MessageFile]) -> None:
    message_list = []

    # Get all the messages till the last user message
    for message in messages:
        message_list.append(message)
        if message.role == "user":
            break

    # Reverse the messages to show the last user message first
    message_list.reverse()

    # Print the user or Assistant messages or images
    for message in message_list:
        for item in message.content:
            # Determine the content type
            if isinstance(item, MessageContentText):
                print(f"{message.role}:\n{item.text.value}\n")
                file_annotations = item.text.annotations
                if file_annotations:
                    for annotation in file_annotations:
                        file_id = annotation.file_path.file_id
                        content = read_assistant_file(file_id)
                        print(f"Annotation Content:\n{str(content)}\n")
            elif isinstance(item, MessageContentImageFile):
                # Retrieve image from file id                
                data_in_bytes = read_assistant_file(item.image_file.file_id)
                # Convert bytes to image
                readable_buffer = io.BytesIO(data_in_bytes)
                image = Image.open(readable_buffer)
                # Resize image to fit in terminal
                width, height = image.size
                image = image.resize((width // 2, height // 2), Image.LANCZOS)
                # Display image
                image.show()

### Process the user messages

In [ ]:
def process_prompt(prompt: str) -> None:
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=prompt)
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please address the user as Jane Doe. The user has a premium account. Be assertive, accurate, and polite. Ask if the user has further questions. "
        + "The current date and time is: "
        + datetime.now().strftime("%x %X")
        + ". ",
    )
    print("processing ...")
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status == "completed":
            # Handle completed
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            print_messages(messages)
            break
        if run.status == "failed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            answer = messages.data[0].content[0].text.value
            print(f"Failed User:\n{prompt}\nAssistant:\n{answer}\n")
            # Handle failed
            break
        if run.status == "expired":
            # Handle expired
            print(run)
            break
        if run.status == "cancelled":
            # Handle cancelled
            print(run)
            break
        if run.status == "requires_action":
            # Handle function calling and continue processing
            pass
        else:
            time.sleep(5)

### Have a conversation with the Assistant

In [ ]:
process_prompt("What is the linear equation when two (x,y) points are (1,1) and (5,10)?")

In [ ]:
process_prompt("I need to solve the equation `3x + 11 = 14`. Can you help me?")

In [ ]:
process_prompt("""x=r*cos(u)sin(v), y=r*sin(u)sin(v), r=2+sin(7*u+5*v) for 0<u<2π and 0<v<π.
Create a graph of the equation z=r*cos(v).""")

In [ ]:
process_prompt("create a csv file with 10 customer names")

In [ ]:
if should_cleanup:
    client.beta.assistants.delete(assistant.id)
    client.beta.threads.delete(thread.id)